In [1]:
import pandas as pd
import numpy as np


# Alpha 1: VOLUME_PRICE_CORRELATION
# 计算公式: (-1 * CORR(RANK(DELTA(LOG(VOLUME), 1)), RANK(((CLOSE - OPE N) / OPEN)), 6))
# 公式: -1 * 过去6天的VOLUME_DELTA与CLOSE_OPEN_RATIO的相关系数
# 该因子用于捕捉短期量价关系,当成交量变化与收盘价相关性较强时,可能预示未来价格将发生变化
def alpha_1(df):
    volume_delta = np.log(df['VOLUME']).diff(1)
    volume_delta_rank = volume_delta.rank(method='average')
    close_open_ratio = (df['CLOSE'] - df['OPEN']) / df['OPEN']
    close_open_ratio_rank = close_open_ratio.rank(method='average')
    corr = volume_delta_rank.rolling(6).corr(close_open_ratio_rank)
    return -1 * corr

In [2]:
data = pd.DataFrame({'VOLUME': [100, 200, 150, 300, 250],
                     'CLOSE': [10, 12, 15, 11, 13],
                     'OPEN': [9, 10, 14, 10, 12]})
print(alpha_1(data))

0   NaN
1   NaN
2   NaN
3   NaN
4   NaN
dtype: float64
